# Decorators and Closures

The idea: **function decorators** let us "mark" functions in the source code to enhance their behaviour in some way. Now, mastering this requires understanding **closures** - which is what we get when functions capture variables defined outside of their bodies. (This reminds me also of lambdas in C++). 

If you want to implement your own function decorators, one needs to understand **closures** and then the need for `nonlocal` becomes obvious. 

The goal of this notebook is to explain:

- how function decorators work from the simplest registration to the rather more complicated parameterized ones. 

### Decorators 

*Def* A decorator is a **callable** that takes another function as an argument (the decorated function). 

The decorator might perform some processing with the decorated function, are returns it or replaces it with another function **or callable object.**

This code ...
```python
@decorate
def target():
    print('running target()')
```

... has the same effect as writing this:
```python
def target():
    print('running target()')

target = decorate(target)
```

In [ ]:
# example 
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco 
def target():
    print("running target()")
    
target() # equivalent to deco(target)()

running inner()


In [5]:
target # equivalent to deco(target)

<function __main__.deco.<locals>.inner()>

### When Python executes decorators

In [ ]:
registry = [] # will hold references to functions decorated with @register

def register(func):
    print(f"running register({func})")
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')
    
@register
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x7a7e187b4ee0>)
running register(<function f2 at 0x7a7e187b5240>)
running register(<function f3 at 0x7a7e187b4dc0>)


In [7]:
if __name__ == '__main__':
    main()

running main()
registry -> [<function f1 at 0x7a7e187b4ee0>, <function f2 at 0x7a7e187b5240>, <function f3 at 0x7a7e187b4dc0>]
running f1()
running f2()
running f3()


What the above snippet of code shows is that a key feature of decorators is that the Python interpreter executes them right after the decorated function is defined. This is usually at *import time*, when a module is loaded by Python.

### Powerful decorators in the standard library 

- `@cache`
- `@lru_cache`
- `@singledispatch`